# 文本预处理

***核心思想***   
如何把词变成可以训练的东西

In [1]:
import collections
import re
import d2l
from d2l import torch as d2l

## 读数据集

In [2]:
#下载一本书 是一个txt文件
d2l.DATA_HUB['time_machine'] = (d2l.DATA_URL + 'timemachine.txt',
                                '090b5e7e70c295757f55df93cb0a180b9691891a')

#将数据集读取到由文本行组成的列表中
def read_time_machine():  
    """Load the time machine dataset into a list of text lines."""
    with open(d2l.download('time_machine'), 'r') as f:
        lines = f.readlines()
    #把非大小写字母的变成空格 lower()变小写
    return [re.sub('[^A-Za-z]+', ' ', line).strip().lower() for line in lines]


# lines为一维列表，每一个元素代表一行文本
lines = read_time_machine()
## text lines: 3221
print(f)
print(lines[0])
print(lines[10])

NameError: name 'f' is not defined

## 翻译成token

In [3]:
#token翻译成词元
def tokenize(lines, token='word'):  
    """将文本行拆分为单词或字符标记。"""
    if token == 'word':
        return [line.split() for line in lines]
    elif token == 'char':
        return [list(line) for line in lines]
    else:
        print('错误：未知令牌类型：' + token)

# 得到tokens为二维列表，表示将原始文本的每一行都切成词元后的结果
tokens = tokenize(lines)
for i in range(11):
    print(tokens[i])

['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
[]
[]
[]
[]
['i']
[]
[]
['the', 'time', 'traveller', 'for', 'so', 'it', 'will', 'be', 'convenient', 'to', 'speak', 'of', 'him']
['was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us', 'his', 'grey', 'eyes', 'shone', 'and']
['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']


## 构建一个字典，通常也叫做词表（vocabulary），用来将字符串标记映射到从0开始的数字索引中

In [4]:
#把不同的token映射到一个从0开始的索引上面
#min_freq 如果一个token出现的次数少于min_freq，就丢掉，就不训练了
class Vocab:  
    """文本词表"""
    def __init__(self, tokens=None, min_freq=0, reserved_tokens=None):
        if tokens is None:
            tokens = []
        if reserved_tokens is None:
            reserved_tokens = []
        # 得到 {词元:出现次数} 形式的字典
        counter = count_corpus(tokens)
        #对token出现次数排序
        #key=lambda x: x[1]代表以x[1]排序，也就是出现次数排序
        #reverse=True是指倒序排序
        self.token_freqs = sorted(counter.items(), key=lambda x: x[1],reverse=True)
        #self.unk是未知token，下标为0.reserved_tokens是保留的词元，保留的词元放在未知词元后面
        self.unk, uniq_tokens = 0, ['<unk>'] + reserved_tokens
        uniq_tokens += [
            token for token, freq in self.token_freqs
            #如果一个token出现的次数少于min_freq，就丢掉，就不训练了，样本个数太少的话，训练也没有意义
            if freq >= min_freq and token not in uniq_tokens]
        self.idx_to_token, self.token_to_idx = [], dict()
        for token in uniq_tokens:
             # 下标:词元 映射中就把该词元加进去，同时更新{词元:对应的下标idx}的字典
            self.idx_to_token.append(token) 
            self.token_to_idx[token] = len(self.idx_to_token) - 1


    #长度是uniq_tokens的个数
    def __len__(self):
        return len(self.idx_to_token)

    #给一个token 返回index
    def __getitem__(self, tokens):
        # # 判断tokens是不是列表或元组，如果是，则返回tokens对应的idx值
        if not isinstance(tokens, (list, tuple)): #判断是否为一个词语
            # 如果token_to_idx里有tokens就返回对应的值，否则返回self.unk
            return self.token_to_idx.get(tokens, self.unk)
        return [self.__getitem__(token) for token in tokens] #如果为句子，则以列表形式返回

    #给index 返回token
    def to_tokens(self, indices):
        if not isinstance(indices, (list, tuple)):#判断是否为1个id
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices] #如果为句子，则以列表形式返回

#这里的tokens是1维或2维列表
def count_corpus(tokens):  
    """统计标记的频率。"""
    ## 一维列表为空 or 是二维列表
    if len(tokens) == 0 or isinstance(tokens[0], list):
        # 将词元列表展平成一个列表
        tokens = [token for line in tokens for token in line]
    #计算每个token出现的次数
    #返回字典形式的key:value统计个数，注意没有排序哦
    return collections.Counter(tokens)

## 构建词汇表

In [13]:
vocab = Vocab(tokens)
print(list(vocab.token_to_idx.items())[:20])
print(len(vocab))

[('<unk>', 0), ('the', 1), ('i', 2), ('and', 3), ('of', 4), ('a', 5), ('to', 6), ('was', 7), ('in', 8), ('that', 9), ('my', 10), ('it', 11), ('had', 12), ('me', 13), ('as', 14), ('at', 15), ('for', 16), ('with', 17), ('but', 18), ('time', 19)]
4580


## 将每一行文本转换成一个数字索引列表

In [23]:
for i in [0, 10]:
    print('words:', tokens[i])
    print('indices:', vocab[tokens[i]])

NameError: name 'tokens' is not defined

In [24]:
#所有内容打包到load_corpus_time_machine函数中
'''在使⽤上述函数时，我们将所有功能打包到load_corpus_time_machine函数中，该函数返回
corpus（词元索引列表）和vocab（时光机器语料库的词表）。我们在这⾥所做的改变是：
    1. 为了简化后⾯章节中的训练，我们使⽤字符（⽽不是单词）实现⽂本词元化；
    2. 时光机器数据集中的每个⽂本⾏不⼀定是⼀个句⼦或⼀个段落，还可能是⼀个单词，因此
    返回的corpus仅处理为单个列表，⽽不是使⽤多词元列表构成的⼀个列表
'''
def load_corpus_time_machine(max_tokens=-1):  
    """返回时光机器数据集的标记索引列表和词汇表。"""
    lines = read_time_machine()
    tokens = tokenize(lines, 'char')
    vocab = Vocab(tokens)
    '''这里等价于
        for line in tokens:
            for token in line:
                corpus.append(vocab[token])
    也就是对于每一行的每个词元，我都去查一下词表，然后添加到corpus里面。
    所以corpus存储的是《时光机器》这本书的词元表示，vocab是词表。
    搞懂这个其实就可以了！'''
    corpus = [vocab[token] for line in tokens for token in line]
    if max_tokens > 0:
        corpus = corpus[:max_tokens]
    return corpus, vocab

corpus, vocab = load_corpus_time_machine()
len(corpus), len(vocab)

NameError: name 'read_time_machine' is not defined

# 答疑

- 模型训练的是数字，不是字符串，所以读到文本，然后拆分成token，映射成下标
- 排序的好处是常见的词排在前面，会对cache好一点
- 需要注意的是，不管是训练还是测试，都只能用一个vocab(词元和下标的对应关系），如果用两个vocab就乱套了。